In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import chardet
import re
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Concatenate, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('data_output/df_exploded_clustered.csv')

In [3]:
df = df[df['Sequence'].apply(lambda x: len(x.split())) >= 2]
df = df.sample(frac=0.4, random_state=42)

In [4]:
df.head()

,Player,Sequence,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,cluster
259263,Stan Wawrinka,5 b3 b3 b3 b3 s3 f1 b3 b3,20160909-M-US_Open-SF-Stan_Wawrinka-Kei_Nishikori,87,0,1,3.0,3.0,40-40,1
491668,John Mcenroe,f28 f1 b2 f3 f1d@,19860826-M-US_Open-R128-John_Mcenroe-Paul_Anna...,122,1,1,1.0,0.0,AD-40,1
259397,Gael Monfils,6 f1 f1w@,20160909-M-US_Open-SF-Novak_Djokovic-Gael_Monfils,35,0,0,5.0,0.0,30-30,2
271179,Milos Raonic,5 f2 v3 v1*,20160125-M-Australian_Open-R16-Stan_Wawrinka-M...,317,2,2,3.0,5.0,0-0,0
141274,Daniel Elahi Galan,6 f1 f3*,20201129-M-Lima_CH-F-Daniel_Elahi_Galan-Thiago...,84,1,0,2.0,5.0,15-0,2


In [5]:
df[['Pt1', 'Pt2']] = df['Pts'].str.split("-", expand=True)
# if value in Pt1 or Pt2 isn't 15, 30, 40, or AD, then replace with TB
df['Pt1'] = df['Pt1'].apply(lambda x: 55 if x not in ['15', '30', '40'] else x)
df['Pt2'] = df['Pt2'].apply(lambda x: 55 if x not in ['15', '30', '40'] else x)

# Objetivo
Determinar que golpes se suelen realizar luego de un golpe en particular. No a modo de devolucion, sino secuencia de golpes de un mismo jugador.

# Preprocesamiento de los datos

In [6]:
context_data = df[['Pt','Pt1', 'Pt2', 'Set1', 'Set2', 'Gm1', 'Gm2', 'cluster']]

scaler = MinMaxScaler()
context_data_normalized = scaler.fit_transform(context_data)

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Sequence'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1 

In [8]:
sequences_tokenized = tokenizer.texts_to_sequences(df['Sequence'])

In [9]:
# Preparar X (golpe actual) e y (siguiente golpe) a partir de secuencias
X = []
y = []
context_features = []

for seq in sequences_tokenized:
    for i in range(len(seq) - 1):
        X.append(seq[i])   # Golpe actual
        y.append(seq[i + 1])  # Siguiente golpe
        context_features.append(context_data_normalized[i])

X = np.array(X).reshape(-1, 1)  # Redimensionar X para que sea una matriz de una columna
y = np.array(y)
X_context = np.array(context_features)

print(f"Verificación")
print(f"Entrada X (golpe actual): {X[0][0]}")
print(f"Salida y (siguiente golpe): {y[0]}")

sequences_tokenized[0]

Verificación
Entrada X (golpe actual): 10
Salida y (siguiente golpe): 2


[10, 2, 2, 2, 2, 9, 1, 2, 2]

Efectivamente el primer golpe es "9" y el siguiente es "2"

# Modelo

In [10]:
# Definir la entrada para el golpe actual
input_golpe = Input(shape=(1,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=32)(input_golpe)
lstm_layer = LSTM(32)(embedding_layer)

# Definir la entrada para las características contextuales
input_context = Input(shape=(X_context.shape[1],))

# Concatenar las dos entradas (golpe actual + contexto)
concat_layer = Concatenate()([lstm_layer, input_context])

# Añadir una capa densa para la predicción del siguiente golpe
output = Dense(vocab_size, activation='softmax')(concat_layer)

# Definir el modelo con las dos entradas
model = Model(inputs=[input_golpe, input_context], outputs=output)

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y testeo (80%-20%)
X_train_golpe, X_test_golpe, X_train_context, X_test_context, y_train, y_test = train_test_split(
    X, X_context, y, test_size=0.2, random_state=42
)

# Entrenar el modelo con las dos entradas
model.fit(
    [X_train_golpe, X_train_context],
    y_train,
    epochs=10,
    batch_size=16,
    validation_data=([X_test_golpe, X_test_context], y_test)
)


Epoch 1/10


C:\Users\agusd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_3']. Received: the structure of inputs=('*', '*')
  warnings.warn(


35927/35927 ━━━━━━━━━━━━━━━━━━━━ 44s 1ms/step - accuracy: 0.1741 - loss: 2.9707 - val_accuracy: 0.1864 - val_loss: 2.8493
Epoch 2/10
35927/35927 ━━━━━━━━━━━━━━━━━━━━ 42s 1ms/step - accuracy: 0.1882 - loss: 2.8404 - val_accuracy: 0.1887 - val_loss: 2.8477
Epoch 3/10
35927/35927 ━━━━━━━━━━━━━━━━━━━━ 43s 1ms/step - accuracy: 0.1902 - loss: 2.8371 - val_accuracy: 0.1885 - val_loss: 2.8484
Epoch 4/10
35927/35927 ━━━━━━━━━━━━━━━━━━━━ 42s 1ms/step - accuracy: 0.1897 - loss: 2.8386 - val_accuracy: 0.1894 - val_loss: 2.8488
Epoch 5/10
35927/35927 ━━━━━━━━━━━━━━━━━━━━ 42s 1ms/step - accuracy: 0.1893 - loss: 2.8366 - val_accuracy: 0.1892 - val_loss: 2.8490
Epoch 6/10
35927/35927 ━━━━━━━━━━━━━━━━━━━━ 42s 1ms/step - accuracy: 0.1884 - loss: 2.8402 - val_accuracy: 0.1902 - val_loss: 2.8514
Epoch 7/10
35927/35927 ━━━━━━━━━━━━━━━━━━━━ 45s 1ms/step - accuracy: 0.1897 - loss: 2.8418 - val_accuracy: 0.1901 - val_loss: 2.8507
Epoch 8/10
35927/35927 ━━━━━━━━━━━━━━━━━━━━ 43s 1ms/step - accuracy: 0.1901 - lo

In [15]:
# Save model
model.save('tenis_rnn_model_one_shot_input.keras')

In [37]:
# Ejemplo de predicción con el golpe 'f2' y algunas características contextuales
input_golpe_test = 'f3'
input_golpe_sequence = tokenizer.texts_to_sequences([input_golpe_test])
input_context_test = context_data_normalized[0]  # Usamos la primera fila de las características normalizadas como ejemplo

# Realizar la predicción
predicted_probabilities = model.predict([np.array(input_golpe_sequence), np.array([input_context_test])])

# Obtener los tres índices con las mayores probabilidades
top_3_indices = predicted_probabilities[0].argsort()[-3:][::-1]  # Ordenar y obtener los 3 índices más altos

# Convertir los índices a los golpes correspondientes
top_3_golpes = [tokenizer.index_word[idx] for idx in top_3_indices]

# Mostrar resultados
print(f"Golpe actual: {input_golpe_test}")
print(f"Los 3 golpes más probables que siguen: {top_3_golpes}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Golpe actual: f3
Los 3 golpes más probables que siguen: ['b3', 'f3', 'f1']


Por ejemplo sirve para entender: Si se juega al medio, el rival logicamente busca jugar al reves.